In [1]:
import tensorflow as tf
import numpy as np

%matplotlib inline
import pylab

import matplotlib.pyplot as plt
import scipy
import scipy.io as sio
from sklearn.model_selection import train_test_split

image_by_pixel = sio.loadmat('combined_img.mat')
label_by_pixel = sio.loadmat('combined_lab.mat')

test_image_by_pixel = sio.loadmat('S0D2_jpgimage.mat')
test_label_by_pixel = sio.loadmat('S0D2_jpglabel.mat')
#test_image_by_pixel = sio.loadmat('s3d1_jpgimage.mat')
# test_label_by_pixel = sio.loadmat('s3d1_jpglabel.mat')

In [2]:
def image_process(image, label, down_sample_rate=4):
    images = image['allimages'][::down_sample_rate ,::down_sample_rate , :]  # downsample the image
    labels = label['disc'][::down_sample_rate ,::down_sample_rate , :]       # downsample the pixel
    image_length1 = images.shape[0]
    image_length2 = images.shape[1]
    num_image = images.shape[2]                # number of images
    image_size = image_length1*image_length2  # image size
    label_size = image_size*2                   # label size  
    label_whole = np.zeros((num_image, label_size))
    # label_whole = np.zeros((num_image, output_size))#the label of whether the image is all trash or not
    images_processed = np.zeros((num_image, image_size))    
    for _ in range(num_image):
        # kk= 1*(np.linalg.norm(labels[:,:,_]) != 0) 
        # if kk ==1:     
        #    label_whole[_,1] = 1               # make output dimension = 2. for example [0,1] or [1,0]
        # else:
        #    label_whole[_,0] = 1
        kk = labels[:,:,_].reshape(image_size)
        for i in range(image_size):
            if kk[i] == 0:
                label_whole[_,i*2] = 1
            else:
                label_whole[_,i*2+1] = 1
        images_processed[_] = images[:,:,_].reshape(image_size)
    return images_processed, label_whole, image_size


output_size = 4096*2                                  # output portal size 1: 0 - garbge 1 - useful
images_processed, label_whole, image_size = image_process(image_by_pixel, label_by_pixel, 4)
test_images_processed, test_label_whole, _ =  image_process(test_image_by_pixel, test_label_by_pixel, 4)


In [3]:
#images_processed.shape
#test_label_whole
output_size
#test_images_processed.shape
#test_label_whole.shape


8192

In [4]:
# define variable placeholder for tf
x = tf.placeholder('float',[None, image_size]) 
y = tf.placeholder('float', [None, output_size])

In [5]:
#label_whole.shape
print(label_whole.shape)
print(images_processed[1].shape)

(552, 8192)
(4096,)


In [6]:
n_nodes_hl1 = 8000
n_nodes_hl2 = 4000
n_nodes_hl3 = 4000
n_nodes_hl4 = 2000
n_nodes_hl5 = 2000
n_nodes_hl6 = 2000
n_nodes_hl7 = 2000
def neural_net_for_whole(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([image_size, n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}
    
    #hidden_4_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_nodes_hl4])),
    #                 'biases':tf.Variable(tf.random_normal([n_nodes_hl4]))}
    
    #hidden_5_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl4, n_nodes_hl5])),
    #                 'biases':tf.Variable(tf.random_normal([n_nodes_hl5]))}
    
    #hidden_6_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl5, n_nodes_hl6])),
    #                  'biases':tf.Variable(tf.random_normal([n_nodes_hl6]))}
    
    #hidden_7_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl6, n_nodes_hl7])),
    #                  'biases':tf.Variable(tf.random_normal([n_nodes_hl7]))}

    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, output_size])),
                    'biases':tf.Variable(tf.random_normal([output_size]))}
    
    
    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
    
    #l4 = tf.add(tf.matmul(l3,hidden_4_layer['weights']), hidden_4_layer['biases'])
    #l4 = tf.nn.relu(l4)
    
    #l5 = tf.add(tf.matmul(l4,hidden_5_layer['weights']), hidden_5_layer['biases'])
    #l5 = tf.nn.relu(l5)
    
    #l6 = tf.add(tf.matmul(l5,hidden_6_layer['weights']), hidden_6_layer['biases'])
    #l6 = tf.nn.relu(l6)
    
    #l7 = tf.add(tf.matmul(l6,hidden_7_layer['weights']), hidden_7_layer['biases'])
    #l7 = tf.nn.relu(l7)

    output = tf.matmul(l3,output_layer['weights']) + output_layer['biases']

    return output

In [7]:
def train_neural_network(x):
    prediction = neural_net_for_whole(x)
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=prediction, labels=y) )
    # double check what cost function to use
    # cost = tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) 
    optimizer = tf.train.AdamOptimizer().minimize(cost)

    hm_epochs = 20 # number of epochs
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            _, epoch_loss = sess.run([optimizer, cost], feed_dict={x: images_processed, y: label_whole})
            #_, epoch_loss = sess.run([optimizer, cost], feed_dict={x: aaa, y: bbb})
            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)
        
        lab = label_whole               # i dont know how to do this in tensorflow, the following are just python
        pred = prediction.eval({x:images_processed})
        print(pred.shape)
        num_image = pred.shape[0]
        N = int(pred.shape[1]/2)
        pred1 = pred.reshape((num_image*N,2))
        wrong = 0
        right = 0
        for i in range(num_image):
            for j in range(N):
                if pred[i,j*2]>pred[i,j*2+1]:
                    if lab[i,j*2] == 1:
                        right += 1
                    else:
                        wrong += 1
                else:
                    if lab[i,j*2] == 0:
                        right += 1
                    else:
                        wrong += 1 
        accuracy = right/(right+wrong)
        print(accuracy)
        print(right)
        print(wrong)
            
        #print(prediction.eval({x:images_processed}), label_whole)   
        #correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        #correct = tf.equal(prediction, y)
        #accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        
        
        #print('Accuracy:',accuracy.eval({x:images_processed, y:label_whole}))
        #print('Accuracy:',accuracy.eval({x:test_images_processed, y:test_label_whole}))
        #print(correct.eval({x:test_images_processed, y:test_label_whole}))
        
            

In [8]:
train_neural_network(x)

Epoch 1 completed out of 20 loss: 1.76611e+08
Epoch 2 completed out of 20 loss: 1.4341e+08
Epoch 3 completed out of 20 loss: 1.16467e+08
Epoch 4 completed out of 20 loss: 9.51831e+07
Epoch 5 completed out of 20 loss: 7.87749e+07
Epoch 6 completed out of 20 loss: 6.6284e+07
Epoch 7 completed out of 20 loss: 5.67823e+07
Epoch 8 completed out of 20 loss: 4.94595e+07
Epoch 9 completed out of 20 loss: 4.36848e+07
Epoch 10 completed out of 20 loss: 3.90079e+07
Epoch 11 completed out of 20 loss: 3.51218e+07
Epoch 12 completed out of 20 loss: 3.182e+07
Epoch 13 completed out of 20 loss: 2.89685e+07
Epoch 14 completed out of 20 loss: 2.64865e+07
Epoch 15 completed out of 20 loss: 2.43205e+07
Epoch 16 completed out of 20 loss: 2.2425e+07
Epoch 17 completed out of 20 loss: 2.07636e+07
Epoch 18 completed out of 20 loss: 1.92997e+07
Epoch 19 completed out of 20 loss: 1.79979e+07
Epoch 20 completed out of 20 loss: 1.68318e+07
(552, 8192)
0.7646134970844656
1728785
532207
